# Substitution

In the prior section, you learned how to build the sympy expressions.

In [7]:
from sympy import *

x = Symbol('x')
f = x**2 + x + 1
f

x**2 + x + 1

However, how to edit the mathematical expression?

You are not able to edit by re-assigning the variable 

In [8]:
x = Symbol('y')
f

x**2 + x + 1

And you are not able to edit the sympy expression directly,
because sympy protects it.

In [12]:
f.args[1] = Symbol('y')

TypeError: 'tuple' object does not support item assignment

In fact, the only ways to make a new formula is:

Destroying the old formula and try to build new formula by scratch.


In [14]:
y = Symbol('y')
y**2 + y + 1

y**2 + y + 1

However, in this way, you can't reuse the old formula programmatically.

The programatic way to replace $x$ by $y$ is substitution

In [17]:
x = Symbol('x')
y = Symbol('y')
g = f.subs(x, y)
g

y**2 + y + 1

Because sympy expressions are immutable, the substitution doesn't change the original expression

In [19]:
f

x**2 + x + 1

It's similar to string `replace` in Python

In [22]:
f = "x**2 + x + 1"
g = f.replace('x', '(y + 1)')
g

'(y + 1)**2 + (y + 1) + 1'

In [23]:
f

'x**2 + x + 1'

Substitution the variable with general function works

(with automatic evaluation)

In [55]:
x = Symbol('x')
y = Symbol('y')
expr = x**2 + x + 1
expr.subs(x, y + 1)

y + (y + 1)**2 + 2

In [56]:
f = Function('f')

expr.subs(x, f(y + 1))

f(y + 1)**2 + f(y + 1) + 1

Multivariate substitution also works
(with dictionary)

In [57]:
expr = f(x, y)
g = Function('g')

expr.subs({x: f(1), y: g(f(1), 2)})

f(f(1), g(f(1), 2))

The source of substitution can be more general than symbols

In [61]:
expr = f(x + 1) + g(y) + 1
h = Function('h')
z = Symbol('z')

expr.subs(f(x + 1), h(z))

g(y) + h(z) + 1

## Pattern Matching

The pattern matching is useful for finding the coffecients for mathematical formula.

For example, if you want to check if the formula is a quadraitic equation,
you can define some pattern expression $ax^2 + bx + c$ using `Wild`.

In [76]:
x = Symbol('x')
a = Wild('a')
b = Wild('b')
c = Wild('c')

pattern = a*x**2 + b*x + c
pattern

x**2*a_ + x*b_ + c_

And you can use `expr.match(pattern)` to find the coefficients for the substitution.

In [78]:
expr = 2*x**2 + 2*x + 3
match = expr.match(pattern)
match

{a_: 2, b_: 2, c_: 3}

It gives a substitution as solution satifying the equation

`pattern.subs(match) == expr`

In [81]:
pattern.subs(match)

2*x**2 + 2*x + 3

We'd note that because of some ambiguity of pattern matching,
it can give some undesired (but correct) answer.

For example, the following match is correct syntactically, however, you wouldn't expect $x$ to appear in the coefficient

In [82]:
expr = x**2 + 2*x + 3
match = expr.match(pattern)
match

{a_: 2/x, b_: x, c_: 3}

In [83]:
pattern.subs(match)

x**2 + 2*x + 3

For this case, you would need to use `exclude` option.

In [85]:
a = Wild('a', exclude=[x])
b = Wild('b', exclude=[x])
c = Wild('c', exclude=[x])
pattern = a*x**2 + b*x + c

match = expr.match(pattern)
match

{b_: 2, a_: 1, c_: 3}

# Numeric Evaluation

### Evaluating $\pi$

In Python math library, you can only get 17 digits of $\pi$

In [108]:
import math

math.pi

3.141592653589793

How do you get 100 digits of $\pi$? or even 1000 digits?

In [106]:
pi.evalf(100)

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117068

## Expand the decimals

In [168]:
for prec in (10, 20, 30, 40):
    display(Rational(100, 9899).evalf(prec))

0.01010203051

0.010102030508132134559

0.0101020305081321345590463683200

0.01010203050813213455904636832003232649763

In [166]:
for prec in (10, 20, 30, 40, 50):
    display(Rational(1000, 9801).evalf(prec))

0.1020304051

0.10203040506070809101

0.102030405060708091011121314152

0.1020304050607080910111213141516171819202

0.10203040506070809101112131415161718192021222324253

## Evaluation by substitution

Since substitution also have evaluation semantics, you can use `subs` with numeric values for convenience.

In fact, if sympy sees that every arguments of functions are floating points, it considers the function as 'numeric' and immediately evaluates to a number.

In [86]:
(sin(x) / x).subs(x, 3.0)

0.0470400026866224

## Irrational Numbers

Unless asked for numeric evaluation, SymPy tries to keep the formula as mathematically precise as possible.

For example, square roots and logarithms are irrational, and if you start to express it into some number, 
you lose some information how the number come from.

For example, take an example of a golden ratio

In [215]:
expr = (1 + sqrt(5)) / 2

In [216]:
expr

1/2 + sqrt(5)/2

If you evaluate it first to number

In [217]:
expr.evalf()

1.61803398874989

Now given that number, can you guess the original number back?

It is not possible because there can be so many plausible guesses, like below.

In [238]:
display(Rational(34, 21).evalf())
display(Rational(233, 144).evalf())
display(Rational(610, 377).evalf())

1.61904761904762

1.61805555555556

1.61803713527851

This example shows that sympy numbers satisfies the mathematical identities exactly,
while `math` library always has precision issues.

In [193]:
sqrt(2) ** 2 - 2

0

In [195]:
math.sqrt(2) ** 2 - 2

4.440892098500626e-16

The math library or numpy library is insufficient to use in actual mathematics, than numeric analysis.

So if you actually want to do math, SymPy is the only option.

Every SymPy functions, can be used to construct arbitrary mathematical constant that can be generated by mathematical formula.

In [ ]:
sin(1)

For example, Apery's constant (https://en.wikipedia.org/wiki/Ap%C3%A9ry%27s_constant) is not available in standard math library. 

But if you need it, you can build it your own by using sympy functions.

In [ ]:
zeta(3).evalf()

# Parsing

We have learned how to build the mathematical formula by programming with symbols.

However, you can alternatively build SymPy expressions by parsing from strings.

The pros of using parser is that:

- It is easy to store expression as file and read to SymPy (serializing/deserializing).
- It can use more 'mathematical' grammar for expressions than Python:
  - You always need to declare symbols first, and undefined program variables don't become symbols
  - You can't multiply by juxtaposition

## SymPy parser

`parse_expr` can be used to parse text into SymPy expressions.

`parse_expr` can do the following:

- Parses `sin, cos, ...` into sympy functions.
- Finds undefined variables in the code like `x, y` and tries to inject symbols with same name

In [2]:
from sympy import *

parse_expr('sin(x + y)')

sin(x + y)

You can notice that factorial can be used directly
(with standard_transformations option)

And with `implicit_multiplication` option, you can multiply by juxtaposition.

In [3]:
from sympy.parsing.sympy_parser import (
    standard_transformations,
    convert_xor, 
    implicit_multiplication)

parse_expr('2x**2 + 3x + 4', transformations=standard_transformations + (implicit_multiplication,))

2*x**2 + 3*x + 4

However, the cons is that even though parsing allows you to create symbols without declaring,

you would anyway need symbols later for computing derivatives, solving, ...
(to be introduced)

For that cases, you have to parse strings again for symbols:

In [4]:
parse_expr('sin(x + y)').diff(parse_expr('x'))

cos(x + y)

# Printing

## Math printers

Although you mostly see the LaTeX typesetted expression here,

It is SymPy's `latex` function that does real work behind

(The magic is done by `_repr_latex_` interface from IPython)

Calling `latex` is useful if you want to copy the typesetted SymPy expressions into your LaTeX article.

In [44]:
latex(exp(sin(x**2)))

'e^{\\sin{\\left(x^{2} \\right)}}'

(Be careful that Python escapes the backslashes if string is represented)

In [52]:
print(latex(exp(sin(x**2))))

e^{\sin{\left(x^{2} \right)}}


And you can use `pprint` (pretty printing) if you still need to typeset the mathematical expressions
with the ascii art (like the graphing calculator).

(However, you would only need `latex` for most of the cases)

In [54]:
pprint(exp(sin(x**2)), use_unicode=False)

    / 2\
 sin\x /
e       


And if you want to print with plaintext:

(`sstr` is name picked because of the conflict with Python builtin)

In [43]:
sstr(exp(sin(x**2)))

'exp(sin(x**2))'

## Code printers

SymPy also have the code printers that can print out mathematical expressions as code.

This is useful if you want to 'bake' mathematical formula in SymPy 
and going to paste in other languages for efficient numeric evaluation.

In [45]:
ccode(exp(sin(x**2)))

'exp(sin(pow(x, 2)))'

In [48]:
jscode(exp(sin(x**2)))

'Math.exp(Math.sin(Math.pow(x, 2)))'

In [47]:
mathematica_code(exp(sin(x**2)))

'Exp[Sin[x^2]]'

In fact, sympy contains a toolchain for compiling the mathematical expressions

into different programming languages and math libraries.

And code printers play a critical role in that

(to be explained)

## Exercises

- Try to print `sin(pi/60)` in LaTeX and copy-paste the result with jupyter markdown or https://katex.org/